## 1. Загрузка данных

Подключим необходимые модули:

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

Загрузим и изучим данные:

In [4]:
# этот блок добавлен, потому что выполнял на ПК Windows, чтобы не ошибиться и не переписывать для работы на серере
path_data = os.path.normpath('/datasets/insurance.csv')

In [5]:
data = pd.read_csv(path_data)

Посмотрим на данные:

In [6]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [8]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные хорошие и не нуждаются в предобработке.

Выполним масштабирование признаков:

In [7]:
numeric = list(data)[:-1]
scaler = StandardScaler()
scaler.fit(data[numeric])
data[numeric] = scaler.transform(data[numeric])

Выделим признаки и целевой признак:

In [8]:
target = data.loc[:, ['Страховые выплаты']]
features = data.drop(['Страховые выплаты'] , axis=1)

## 2. Умножение матриц

Нужно ответить на вопрос, изменится ли качество линейной регрессии, при умножении признаков на обратимую матрицу.

Модель линейной регрессии работает следующим образом:

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Тогда предсказания будут иметь вид:

$$
a = X(X^T X)^{-1} X^T y
$$

При умножении на обратимую матрицу предсказания имеют вид:

$$
a = XPw
$$

Тогда формула обучения:

$$
w = ((XP)^T (XP))^{-1} (XP)^T y = P^{-1}X^{-1}(P^{T}X^{T})^{-1}P^{T}X^{T}y
$$

Раскроем скобки:

$$
w = P^{-1}X^{-1}(X^{T})^{-1}(P^{T})^{-1}P^{T}X^{T}y
$$

Сократим умножение транспонированной матрицы *P* на обратную ей, т.к. в итоге получится единичная матрица, которая при умножении на любую другую даст любую другую:

$$
w = P^{-1}(X^T X)^{-1} X^T y
$$

Тогда предсказания будут иметь вид:

$$
a = XPP^{-1}(X^T X)^{-1} X^T y = X(X^T X)^{-1} X^T y
$$

**Ответ:** качество модели при умножении признаков на обратимую матрицу **не изменится**.

**Обоснование:** при умножении признаков на обратимую матрицу, каждый признак объекта будет содержать информацию из всех своих признаков, взвешенных на столбец случайной обратимой матрицы. Есть 2 важных момента:
1. Признаки разных объектов не перемешаются
2. В пределах объектов, признаки взвешиваются одинаково и однозначно (определяется обратимой матрицей и содержат всю информацию из нее)


## 3. Алгоритм преобразования

**Алгоритм**: 
0. Масштабирование признаков
1. Генерирование случайной обратимой матрицы размерностью *n x n*, где *n* - количество признаков
2. Умножение матрицы признаков на сгенерированную матрицу
3. На полученных данных обучение модели 

**Обоснование**

После умножения признаков на обратимую матрицу, признаки не перемешиваются между объектами, а в пределах объектов претерпевают одинаковые преобразования, так что это позволяет защитить данные клиентов страховой компании, без влияния на качество модели линейной регрессии.

## 4. Проверка алгоритма

Создадим случайную обратимую матрицу. Чтобы сохранить исходную размерность, матрица должна иметь размерность (4 x 4).

In [9]:
random_matrix = np.random.normal(size=(4,4))
random_matrix_inv = np.linalg.inv(random_matrix)

In [10]:
features_transformed = np.dot(features.values, random_matrix)

Создадим, обучим и проверим кросс-валидацией две модели линейной регрессии. Одну на исходных признаках, другую на признаках, умноженных на обратимую матрицу:

In [11]:
model_original = LinearRegression()
model_transform = LinearRegression()
r2_original = cross_val_score(model_original, features, target, cv=5, scoring='r2').mean()
r2_transform = cross_val_score(model_transform, features_transformed, target, cv=5, scoring='r2').mean()
print('Значение метрики R2 модели, обученной, на исходных признаках: {:.3}, модели, обученной на признаках, \
умноженных на обратимую матрицу: {:.3}'.format(r2_original, r2_transform))

Значение метрики R2 модели, обученной, на исходных признаках: 0.423, модели, обученной на признаках, умноженных на обратимую матрицу: 0.423


## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования